In [32]:
class PicoMotor:
    
    _MIN_SPEED = 50
    _MAX_SPEED = 100
    _MIN_DIFF = 10
    
    def __init__(self, serial_port):
        self.serial_port = serial_port
        
    def _send(self, left, right):
        cmd = f'MOTOR:left:{left}:right:{right}'.encode()
        serial_port.write(cmd)
        
    def _bounding_speed(self, speed):
        if speed == 0:
            return speed
        
        if abs(speed) < self._MIN_SPEED:
            print(f'[PicoMotor]: Lowest speed is {self._MIN_SPEED}, reset value {speed} to it.')
            return self._MIN_SPEED if speed > 0 else -self._MIN_SPEED
        
        if abs(speed) > self._MAX_SPEED:
            print(f'[PicoMotor]: Max speed is {self._MAX_SPEED}, reset value {speed} to it.')
            return self._MAX_SPEED if speed > 0 else -self._MAX_SPEED
        
        return speed
    
    def _get_diff_speed(self, speed, diff):
        if speed >= 0:
            return speed - diff
        
        return speed + diff
    
    def _is_need_change_to_rotate(self, speed, diff_speed):
        return abs(speed) == self._MIN_SPEED or (abs(speed) - abs(diff_speed)) < self._MIN_DIFF
        
    def go(self, speed, diff):
        '''
        high level，會依照 speed, diff 的值來決定要 直走 還是 轉彎
        speed:
            + : forward
            - : backward
        diff:
            + : right
            - : left
        '''
        if diff == 0:
            return self.go_straight(speed)
        
        return self.go_turn(speed, diff)
        
    def go_straight(self, speed):
        '''
        Always 直走
        speed:
            + : forward
            - : backward
        '''
        print(f'[PicoMotor]: go_straight, speed={speed}')
        speed = self._bounding_speed(speed)
        self._send(speed, speed)
        
    def go_turn(self, speed, diff):
        '''
        轉彎專用
        speed:
            + : forward
            - : backward
        diff:
            + : right
            - : left
        '''
        print(f'[PicoMotor]: go_turn, speed={speed}, diff={diff}')
        if speed == 0:
            return
        
        speed = self._bounding_speed(speed)
        diff_speed = self._bounding_speed(self._get_diff_speed(speed, abs(diff)))
        
        if self._is_need_change_to_rotate(speed, diff_speed):
            if (speed > 0 and diff >= 0) or (speed < 0 and diff < 0):
                print('[PicoMotor]: change to right_rotate')
                self.right_rotate()
            else:
                print('[PicoMotor]: change to left_rotate')
                self.left_rotate()
        else:    
            if diff >= 0:
                self._send(speed, diff_speed)
            else:
                self._send(diff_speed, speed)

    def turn_left(self, speed, diff):
        '''
        明確左轉使用
        speed: 
            + : forward
            - : backward
        diff:
            always +
        '''
        print(f'[PicoMotor]: turn_left, speed={speed}, diff={diff}')
        return self.go_turn(speed, -diff)

    def turn_right(self, speed, diff):
        '''
        明確右轉使用
        speed: 
            + : forward
            - : backward
        diff:
            always +
        '''
        print(f'[PicoMotor]: turn_tight, speed={speed}, diff={diff}')
        return self.go_turn(speed, diff)

    def left_rotate(self):
        '''
        原地向左旋轉
        '''
        print(f'[PicoMotor]: left_rotate')
        self._send(-self._MIN_SPEED, self._MIN_SPEED)
        
    def right_rotate(self):
        '''
        原地向右旋轉
        '''
        print(f'[PicoMotor]: right_rotate')     
        self._send(self._MIN_SPEED, -self._MIN_SPEED)
        
    def stop(self):
        self._send(0, 0)
    
    

In [ ]:
import serial

_DEFAULT_PORT = '/dev/ttyTHS1'
_BAUDRATE = 115200

serial_port = serial.Serial(
    port=port,
    baudrate=_BAUDRATE,
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
)

In [15]:
class FakeSerial:
    def write(self, cmd):
        print(f'[FakeSerial]: {cmd}')
        
serial_port = FakeSerial()

In [33]:
m = PicoMotor(serial_port)
print('==== Test forward ====')
m.go_straight(100)
m.go_straight(60)
m.go_straight(20)
print('\n==== Test backward ====')
m.go_straight(-100)
m.go_straight(-60)
m.go_straight(-20)
print('\n==== Test left ====')
m.turn_left(100, 10)
m.turn_left(60, 10)
m.turn_left(60, 20)
m.turn_left(50, 10)
m.turn_left(59, 10)
m.turn_left(30, 10)
print('\n==== Test left ====')
m.go_turn(100, -10)
m.go_turn(60, -10)
m.go_turn(60, -20)
m.go_turn(50, -10)
m.go_turn(59, -10)
m.go_turn(30, -10)
print('\n==== Test right ====')
m.turn_right(100, 10)
m.turn_right(60, 10)
m.turn_right(60, 20)
m.turn_right(50, 10)
m.turn_right(59, 10)
m.turn_right(30, 10)
print('\n==== Test right ====')
m.go_turn(100, 10)
m.go_turn(60, 10)
m.go_turn(60, 20)
m.go_turn(50, 10)
m.go_turn(59, 10)
m.go_turn(30, 10)

==== Test forward ====
[PicoMotor]: go_straight, speed=100
[FakeSerial]: b'MOTOR:left:100:right:100'
[PicoMotor]: go_straight, speed=60
[FakeSerial]: b'MOTOR:left:60:right:60'
[PicoMotor]: go_straight, speed=20
[PicoMotor]: Lowest speed is 50, reset value 20 to it.
[FakeSerial]: b'MOTOR:left:50:right:50'

==== Test backward ====
[PicoMotor]: go_straight, speed=-100
[FakeSerial]: b'MOTOR:left:-100:right:-100'
[PicoMotor]: go_straight, speed=-60
[FakeSerial]: b'MOTOR:left:-60:right:-60'
[PicoMotor]: go_straight, speed=-20
[PicoMotor]: Lowest speed is 50, reset value -20 to it.
[FakeSerial]: b'MOTOR:left:-50:right:-50'

==== Test left ====
[PicoMotor]: turn_left, speed=100, diff=10
[PicoMotor]: go_turn, speed=100, diff=-10
[FakeSerial]: b'MOTOR:left:90:right:100'
[PicoMotor]: turn_left, speed=60, diff=10
[PicoMotor]: go_turn, speed=60, diff=-10
[FakeSerial]: b'MOTOR:left:50:right:60'
[PicoMotor]: turn_left, speed=60, diff=20
[PicoMotor]: go_turn, speed=60, diff=-20
[PicoMotor]: Lowest spe

In [34]:
import ipywidgets.widgets as widgets
from IPython.display import display

speed_slider = widgets.IntSlider(description='speed', min=-100, max=100, step=1, orientation='vertical')
direct_slider = widgets.IntSlider(description='direct', min=-100, max=100, step=1, orientation='horizontal')

slider_container = widgets.HBox([speed_slider, direct_slider])
display(slider_container)

[PicoMotor]: go_straight, speed=-2
[PicoMotor]: Lowest speed is 50, reset value -2 to it.
[FakeSerial]: b'MOTOR:left:-50:right:-50'
[PicoMotor]: go_straight, speed=-8
[PicoMotor]: Lowest speed is 50, reset value -8 to it.
[FakeSerial]: b'MOTOR:left:-50:right:-50'
[PicoMotor]: go_straight, speed=-41
[PicoMotor]: Lowest speed is 50, reset value -41 to it.
[FakeSerial]: b'MOTOR:left:-50:right:-50'
[PicoMotor]: go_straight, speed=-47
[PicoMotor]: Lowest speed is 50, reset value -47 to it.
[FakeSerial]: b'MOTOR:left:-50:right:-50'
[PicoMotor]: go_straight, speed=-65
[FakeSerial]: b'MOTOR:left:-65:right:-65'
[PicoMotor]: go_straight, speed=-70
[FakeSerial]: b'MOTOR:left:-70:right:-70'
[PicoMotor]: go_straight, speed=-73
[FakeSerial]: b'MOTOR:left:-73:right:-73'
[PicoMotor]: go_straight, speed=-76
[FakeSerial]: b'MOTOR:left:-76:right:-76'
[PicoMotor]: go_straight, speed=-77
[FakeSerial]: b'MOTOR:left:-77:right:-77'
[PicoMotor]: go_straight, speed=-79
[FakeSerial]: b'MOTOR:left:-79:right:-79'


[PicoMotor]: go_turn, speed=-27, diff=-100
[PicoMotor]: Lowest speed is 50, reset value -27 to it.
[PicoMotor]: change to right_rotate
[PicoMotor]: right_rotate
[FakeSerial]: b'MOTOR:left:50:right:-50'
[PicoMotor]: go_turn, speed=-25, diff=-100
[PicoMotor]: Lowest speed is 50, reset value -25 to it.
[PicoMotor]: change to right_rotate
[PicoMotor]: right_rotate
[FakeSerial]: b'MOTOR:left:50:right:-50'
[PicoMotor]: go_turn, speed=-24, diff=-100
[PicoMotor]: Lowest speed is 50, reset value -24 to it.
[PicoMotor]: change to right_rotate
[PicoMotor]: right_rotate
[FakeSerial]: b'MOTOR:left:50:right:-50'
[PicoMotor]: go_turn, speed=-22, diff=-100
[PicoMotor]: Lowest speed is 50, reset value -22 to it.
[PicoMotor]: change to right_rotate
[PicoMotor]: right_rotate
[FakeSerial]: b'MOTOR:left:50:right:-50'
[PicoMotor]: go_turn, speed=-21, diff=-100
[PicoMotor]: Lowest speed is 50, reset value -21 to it.
[PicoMotor]: change to right_rotate
[PicoMotor]: right_rotate
[FakeSerial]: b'MOTOR:left:50:r

In [35]:
m = PicoMotor(serial_port)

def on_value_change(v):
    speed = speed_slider.value
    diff = direct_slider.value
    
    m.go(speed, diff)

speed_slider.observe(on_value_change, names='value')
direct_slider.observe(on_value_change, names='value')